<a href="https://colab.research.google.com/github/JohnSili/NovosibCFP/blob/main/%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%2B_%D0%B2%D1%8B%D0%B2%D0%BE%D0%B4_%D0%B3%D1%80%D0%B0%D1%84%D0%B8%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/last_combo.csv')

In [3]:
df = df.drop(['MessageReceivedDate','MessageProcessedDate',	'DateEvent', 'Unnamed: 0'], axis=1)

In [4]:
df['ts'] = pd.to_datetime(df['ts'])
# Извлечение дня недели, часа и минуты
df['Weekday_reg'] = df['ts'].dt.weekday
df['Hour_reg'] = df['ts'].dt.hour
df['Minute_reg'] = df['ts'].dt.minute

In [5]:
features = ['departure_terminal_encoded','checkin_terminal_encoded','Weekday_reg', 'Hour_reg','Minute_reg', 'DayTime','local_or_transfer_binary','config']
target = ['VolumePerMinute','pax_arrival_profile']

X = df[features]
y = df[target]

In [7]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели для прогнозирования y
y_model = RandomForestRegressor(random_state=42)
y_model.fit(X_train, y_train)
y_pred_train = y_model.predict(X_train)
y_pred_test = y_model.predict(X_test)

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
# Рассчитываем метрики
mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

MAE: 11.986119996732139
MSE: 633.1554951723369
RMSE: 25.16258125018848


In [47]:
from sklearn.ensemble import IsolationForest

# Создание и обучение модели для поиска аномалий в X_test
anomaly_model_test = IsolationForest(contamination=0.05, random_state=42)
anomaly_pred_test = anomaly_model_test.fit_predict(X_test.iloc[[0]])
anomalies_X_test = X_test.iloc[[0]][anomaly_pred_test == -1]

# Расчет абсолютных разниц между предсказанными и истинными значениями
absolute_diff = abs(y_pred_test - y_test).sum(axis=1)

# Рассчитываем сумму абсолютных разниц для каждой аномальной строки
anomalies_influence = absolute_diff[anomalies_X_test.index]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [36]:
print(anomalies_influence)

,departure_terminal_encoded,checkin_terminal_encoded,Weekday_reg,Hour_reg,Minute_reg,DayTime,local_or_transfer_binary,config
2037167,0,0,2,16,5,2,1,158


In [ ]:
from sklearn.ensemble import IsolationForest

# Создание и обучение модели для поиска аномалий в X_test
anomaly_model_test = IsolationForest(contamination=0.05, random_state=42)
anomaly_pred_test = anomaly_model_test.fit_predict(X_test)
anomalies_X_test = X_test[anomaly_pred_test == -1]

# Расчет абсолютных разниц между предсказанными и истинными значениями
absolute_diff = abs(y_pred_test - y_test).sum(axis=1)

# Рассчитываем сумму абсолютных разниц для каждой аномальной строки
anomalies_influence = absolute_diff[anomalies_X_test.index]

import matplotlib.pyplot as plt

# Перебираем первые 5 аномальных строки
for idx in anomalies_influence.head(5).index:
    # Получаем данные x для этой аномалии
    anomaly_x = X_test.loc[idx]
      # Исключаем влияние столбца 'config'
    feature_influence['config'] = 0

    # Рассчитываем влияние признаков на итоговый результат y для этой аномалии
    feature_influence = y_model.feature_importances_ * anomaly_x

    # Визуализируем влияние признака
    plt.figure(figsize=(10, 6))
    plt.plot(feature_influence, label=f'Аномалия в строке {idx}')
    plt.xlabel('Время')
    plt.ylabel('Влияние признака')
    plt.title(f'Влияние признака на предсказание для аномалии в строке {idx}')
    plt.legend()
    plt.show()

In [ ]:
!pip install joblib

In [ ]:
import joblib

# Сохранение модели для прогнозирования y
joblib.dump(y_model, '/content/drive/MyDrive/train_dataset_luggage/модельки/y_model.pkl')


['/content/drive/MyDrive/train_dataset_luggage/модельки/anomaly_model.pkl']

In [25]:
import pickle

# Сохранение модели для прогнозирования y
with open('/content/drive/MyDrive/train_dataset_luggage/модельки/y_model_pickle.pkl', 'wb') as file:
    pickle.dump(y_model, file)

